# 銀イルカの3人のセリフをスクレイピングして，同じところと違うところを

とりま wikiとかから智絵里のセリフをスクレイピングする． 

テキスト解析 -> 出現頻度，


セリフをword2vecで学習して，分散表現を計算できるようにする

分散表現を次元削減して2次元座標上にプロットすることで，単語ごとの類似度を可視化する

銀イルカ3人のセリフをスクレイピング  

doc2vecで学習して，

### requirementis

- python 3.7.1
- pipenv
- mecab
    - ipa
    - neologd
    
usage: `pipenv install`  
python, pipenv, pyenv, mecabのインストール方法は省略

In [1]:
import glob
import re
import time
import random
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 20 

## デレステセリフのスクレイピング

In [2]:
def get_quotes_sute(q):
    """
    デレステで公開されている各アイドルのセリフデータを取得する
    [arg]
    q: int
    [ret]
    df: DataFrame
    """
    html = f"https://starlight.kirara.ca/char/{q}"
    r = requests.get(html)
    r.encoding = r.apparent_encoding
    soup = BeautifulSoup(r.text, "lxml")
    texts = soup.find_all("span")
    corpus = []
    for text in texts:
        corpus.append(text.text)
    df = pd.DataFrame(corpus)
    #df[0].value_counts().head(10) # 値の出現頻度をカウント
    df = df[~df.duplicated()] # 重複行をdataframeから除外
    df = df[~df[0].str.match("^[\s|a-zA-Z|<|\+|\d$]")] # 正規表現にマッチするものを除外
    df = df[df[0] != ""]
    return df.reset_index(drop=True)

## モバマスセリフのスクレイピング

In [3]:
def get_quotes_moba(query="緒方智絵里", debug=False):
    """
    モバマスで公開されているセリフ（含 アイドル名、カード名 など）
    をスクレイピングして pandas dataframe にする
    
    [args]
    query: str 省略すると 緒方智絵里 のセリフをスクレイピングする
    debug: Bool Trueにするとスクレイピング数を上限2にする。それだけ もっと良い使い方あるはず...
    [ret]
    df: DataFrame
    """
    
    html = f"https://icws.indigo-bell.com/search?q={query}&st=n"
    r = requests.get(html)
    r.encoding = r.apparent_encoding
    soup = BeautifulSoup(r.text, "lxml")
    a = soup.find("a", string="Last　»").get("href")
    if debug == True:
        n = 2
    else:
        n = int(re.search(r"page=(\d+)&", a).group(1)) # クロールするページ数
    
    df = pd.DataFrame()
    for i in range(n): # セリフが登録されてるページ分ループする
        print(f"crawling page: {i + 1}/{n}")
        html = f"https://icws.indigo-bell.com/search?page={i + 1}&q={query}&st=n"
        r = requests.get(html)
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, "lxml")
        table = soup.find_all("tbody",class_="result")[0]
        rows = table.find_all("tr")
        tmp = []
        tm = 5 + random.randint(0, 5)

        for row in rows:
            cell_txt = []
            for cell in row.find_all(["td", "th"]):
                cell_txt.append(cell.text)
            tmp.append(cell_txt)
        df = df.append(tmp)
        #print(f"sleep: {tm}")
        if i != (n - 1):
            time.sleep(tm)
        else:
            continue
    return df.reset_index(drop=True) # indexを振り直してもとのindexも削除

In [4]:
# deresute id
# chieri : 112
# minami : 185
# yui    : 246
# sae    : 132
# hotaru : 161
# yoshino: 262

In [5]:
idol = ["chieri", "112", "緒方智絵里"]

# data/に idol_moba.csv idol_dere.csvが両方なかったらスクレイピングを実行する
# あったらdataframeにする
if not glob.glob(f"data/{idol[0]}_moba.csv") and not glob.glob(f"data/{idol[0]}_dere.csv"):
    print("nai")
    aa = get_quotes_sute(int(idol[1]))
    bb = get_quotes_moba(query=idol[2])
    aa.to_csv(f"data/{idol[0]}_moba.csv")
    bb.to_csv(f"data/{idol[0]}_dere.csv")
else:
    print("aru")
    aa = pd.read_csv(f"data/{idol[0]}_moba.csv")
    bb = pd.read_csv(f"data/{idol[0]}_dere.csv")

nai
crawling page: 1/20
crawling page: 2/20
crawling page: 3/20
crawling page: 4/20
crawling page: 5/20
crawling page: 6/20
crawling page: 7/20
crawling page: 8/20
crawling page: 9/20
crawling page: 10/20
crawling page: 11/20
crawling page: 12/20
crawling page: 13/20
crawling page: 14/20
crawling page: 15/20
crawling page: 16/20
crawling page: 17/20
crawling page: 18/20
crawling page: 19/20
crawling page: 20/20


## モバ，ステの結合と正規化

In [6]:
bb[3] = bb[3].str.replace("○○", "") # OO (プロデューサー名)部分を落とす
bb[3] = bb[3].str.replace("〇〇", "") # OO (プロデューサー名)部分を落とす
# aa + bb のコーパスを作る
docs = pd.Series()
docs = pd.concat([aa[0], bb[3]])
docs = docs.str.replace("プロデューサー", "PRODUCER")
docs = docs.str.replace("[Producer]", "PRODUCER", regex=False)
docs = docs.reset_index(drop=True)

In [7]:
docs.to_csv(f'data/{idol[0]}_corpus.csv')

/home/wani/sandbox/idol_info/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


## 形態素解析

mecabを使う。インストールしていなければめかぶをインストール

neeologdまでインストールする手順はの参考は
- [ubuntu 18.10 に mecab をインストール](https://qiita.com/ekzemplaro/items/c98c7f6698f130b55d53)
- [WSLでMeCabとJupyter Notebookを動かす](https://qiita.com/hironobukawaguchi3/items/61a8664dd26c1da66880)
- [MecabをPythonで使うまで](https://qiita.com/Sak1361/items/47e9ec464ccc770cd65c)

---

$ mecab -d /usr/lib/mecab/dic/mecab-ipadic-neologd

In [24]:
mecab_path = !echo `mecab-config --dicdir`
def extractNoun(text,
                mecab_mode = rf'-Owakati -d {mecab_path[0]}/mecab-ipadic-neologd',
                noun="名詞"):
    """
    [arg]: sentence
    [ret]: a list of noun words
    """
    print(mecab_mode)
    tagger = MeCab.Tagger(mecab_mode)
    node = tagger.parseToNode(text)
    keywords = []
    while node:
        surface = node.surface # 表層形を表示(str)
        meta = node.feature.split(",") # ['記号', '読点', '*', '*', '*', '*', '、', '、', '、']
        if meta[0] == noun:
            keywords.append(node.surface)
        node = node.next # 次のノードのアドレスを指定
    return keywords

def tokenize(text, wakati=True, dic="neologd"):
    """
    テキストを形態素解析して分かち書きする
    [arg]
    text: str
    [ret]
    term: str
    """
    import MeCab
    if dic == "neologd":
        mecab_mode = MeCab.Tagger(rf'-Owakati -d {mecab_path[0]}/mecab-ipadic-neologd') # debugging
    else:
        mecab_mode = MeCab.Tagger(r'-Owakati')
    if wakati == True: # 分かち書きしたものをそのまま取り出す
        term = mecab_mode.parse(text).strip()
    else: # 品詞で
        term = extractNoun(text, mecab_mode)
    return term

In [25]:
import MeCab
text = "テキストを形態素解析して分かち書きする"
print(extractNoun(text))

-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd


RuntimeError: 

In [28]:
mecab = MeCab.Tagger(r"-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")

print(mecab.parse("私はペンを持っています。"))

RuntimeError: 

In [ ]:
print(f'ipa-dic    ： {tokenize("特急はくたか", dic = "ipa")}')
print(f'neologd-dic： {tokenize("特急はくたか")}')

In [ ]:
from gensim import corpora, models

In [ ]:
docs_list = [tokenize(row).split() for row in docs]
dic = corpora.Dictionary(docs_list)
dic.save_as_text('./data/text.dict')  # 保存
#gensim.corpora.Dictionary.load_from_text('./data/text.dict')  # 次回からファイルロード可

In [ ]:
print(len(dic)) # dictionaryに登録された単語(形態素)数

In [ ]:
# 単語出現頻度を数える
# 余談: bashだと $ sort -t$'\t' -k3 -nr text.dict で単語とその出現頻度回数カウントできる
from collections import defaultdict
freq = defaultdict(int)

for text in docs_list:
    for token in text:
        freq[token] += 1
print(sorted(freq.items(), key=lambda x:x[1], reverse=True))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
corpus = [dic.doc2bow(doc) for doc in docs_list] # コーパスの作成
corpora.MmCorpus.serialize('./data/text.mm', corpus)  # 保存

In [ ]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

# 折角計算したのでpickleに保存
import pickle
with open('./data/corpus_tfidf.dump', mode='wb') as f:
    pickle.dump(corpus_tfidf, f)

# 次回からはロードできます
# with open('./data/corpus_tfidf.dump', mode='rb') as f:
#     corpus_tfidf = pickle.load(f)

In [ ]:
tfidf.idfs

In [ ]:
# 文書量にもよりますが、数時間程度かかったりもします。
# '18/12/03追記: LdaMulticore で worker の数を増やせばかなり早くなるかもです
lda = models.LdaModel(corpus=corpus_tfidf, id2word=dic,
                             num_topics=50, minimum_probability=0.001,
                             passes=20, update_every=0, chunksize=10000)
lda.save('./data/lda.model')  # 保存
#lda = gensim.models.LdaModel.load('./data/lda.model')  # 次回からロード可

In [ ]:
lda.print_topic(10)

In [ ]:
for i in range(10):
    print('tpc_{0}: {1}'.format(i, lda.print_topic(i)))